In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix

from ecephys.graham import paths
from ecephys.utils.utils import load_df_h5
from ecephys.scoring import load_visbrain_hypnogram

In [3]:
pd.options.mode.chained_assignment = None  # default='warn'

## Read the data

In [7]:
condition = "BL"
subject = "Segundo"

hypnogram_path = Path(paths.hypnograms[condition][subject])
hypnogram = load_visbrain_hypnogram(hypnogram_path)

ripple_path = Path(paths.ripples[condition][subject])
with pd.HDFStore(ripple_path) as store:
    ripples, metadata = load_df_h5(store)
    
bl_hypno = hypnogram
ripples_bl = ripples

In [8]:
condition = "SR"
subject = "Segundo"

hypnogram_path = Path(paths.hypnograms[condition][subject])
hypnogram = load_visbrain_hypnogram(hypnogram_path)

ripple_path = Path(paths.ripples[condition][subject])
with pd.HDFStore(ripple_path) as store:
    ripples, metadata = load_df_h5(store)
    
rec_hypno = hypnogram
ripples_rec = ripples

## Look at basic summary statistics

In [9]:
bl_total_NREM = bl_hypno.loc[bl_hypno['state'] == 'N2'].duration.sum()
bl_total_NREM

2896.0

In [10]:
rec_total_NREM = rec_hypno.loc[rec_hypno['state'] == 'N2'].duration.sum()
rec_total_NREM

3797.9899864196777

In [11]:
len(ripples_bl) / bl_total_NREM

0.39917127071823205

In [12]:
len(ripples_rec) / rec_total_NREM

0.33438740084652374

In [13]:
ripples_bl.median()

start_time            3969.845400
end_time              3969.909600
duration                 0.068400
center_time           3969.877500
nadir_time            3969.880000
envelope_integral    37242.903962
envelope_peak          355.583101
mean_rms                47.433510
summed_rms             521.768613
max_rms                 63.438855
mean_amplitude         231.432421
max_amplitude          311.399042
dtype: float64

In [14]:
ripples_rec.median()

start_time            3670.793800
end_time              3670.851800
duration                 0.065600
center_time           3670.822800
nadir_time            3670.825200
envelope_integral    35719.235369
envelope_peak          367.307960
mean_rms                48.129624
summed_rms             529.425863
max_rms                 66.519562
mean_amplitude         235.773843
max_amplitude          328.463406
dtype: float64

In [15]:
ripples_bl.mean()

start_time            4094.019284
end_time              4094.094094
duration                 0.074809
center_time           4094.056689
nadir_time            4094.053493
envelope_integral    40804.593608
envelope_peak          377.827454
mean_rms                49.394707
summed_rms             543.341779
max_rms                 65.952794
mean_amplitude         245.213372
max_amplitude          328.457268
dtype: float64

In [16]:
ripples_rec.mean()

start_time            3730.355360
end_time              3730.426017
duration                 0.070658
center_time           3730.390689
nadir_time            3730.389108
envelope_integral    39229.908841
envelope_peak          380.591924
mean_rms                49.620235
summed_rms             545.822587
max_rms                 68.668772
mean_amplitude         245.678872
max_amplitude          340.281439
dtype: float64

## Look at feature distributions and correlations

In [17]:
ax = scatter_matrix(ripples_bl, figsize=(10, 10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
ax2 = scatter_matrix(ripples_bl.applymap(np.log), figsize=(10, 10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Look at log-normally distributed features

In [19]:
import scipy.stats as stats

def plot_qq(bl, rec, var):
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    
    stats.probplot(bl[var], dist="norm", plot=axes[0, 0])
    axes[0, 0].set_title("Baseline {}".format(var))
    stats.probplot(np.log(bl[var]), dist="norm", plot=axes[0, 1])
    axes[0, 1].set_title("Baseline log {}".format(var))
    stats.probplot(rec[var], dist="norm", plot=axes[1, 0])
    axes[1, 0].set_title("Recovery {}".format(var))
    stats.probplot(np.log(rec[var]), dist="norm", plot=axes[1, 1])
    axes[1, 1].set_title("Recovery log {}".format(var))

    plt.show()
            
plot_qq(ripples_bl, ripples_rec, 'duration')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

def plot_acf_pacf(bl, rec, var, lags=None):
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))

    plot_acf(bl[var], lags=lags, ax=axes[0, 0], title="Baseline {} acf".format(var))
    plot_pacf(bl[var], lags=lags, ax=axes[0, 1], title="Baseline {} pacf".format(var))
    plot_acf(rec[var], lags=lags, ax=axes[1, 0], title="Recovery {} acf".format(var))
    plot_pacf(rec[var], lags=lags, ax=axes[1, 1], title="Recovery {} pacf".format(var))

    plt.show()

In [21]:
plot_acf_pacf(ripples_bl, ripples_rec, 'duration')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
def plot_vs_time(bl, rec, var):
    fig, axes = plt.subplots(2, 1)
    bl.plot.scatter(x='center_time', y=var, ax=axes[0], title='Baseline')
    rec.plot.scatter(x='center_time', y=var, ax=axes[1], title='Recovery')
    plt.show()

In [23]:
plot_vs_time(ripples_bl, ripples_rec, 'duration')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
def test_slope(ripples, x, y):
    return stats.linregress(ripples[x], ripples[y])

In [25]:
test_slope(ripples_bl, 'center_time', 'duration')

LinregressResult(slope=-4.648940652712044e-09, intercept=0.07482837558713902, rvalue=-0.00029910715296810293, pvalue=0.991894716302955, stderr=4.575349551058696e-07)

In [26]:
test_slope(ripples_rec, 'center_time', 'duration')

LinregressResult(slope=-3.577787755402903e-07, intercept=0.0719922924080852, rvalue=-0.02937339884497676, pvalue=0.2955718904426008, stderr=3.4191118603936886e-07)

In [27]:
stats.ttest_ind(np.log(ripples_bl.duration), np.log(ripples_rec.duration), equal_var=False)

Ttest_indResult(statistic=4.2334704315620915, pvalue=2.3891171502544685e-05)

In [28]:
# https://dfrieds.com/math/effect-size.html
def interpret_cohens_d(cohens_d):
    """
    Determines text interpretation of effect size given Cohen's d value

    :param cohens_d: float of Cohen's d value
    :returns: effect_size_interpretation: adjective to describe magnitude of effect size
    """
    if 0<=cohens_d<0.1:
        effect_size_interpretation = "Very Small"
    elif 0.1<=cohens_d<0.35:
        effect_size_interpretation = "Small"
    elif 0.35<=cohens_d<0.65:
        effect_size_interpretation = "Medium"
    elif 0.65<=cohens_d<0.9:
        effect_size_interpretation = "Large"
    elif cohens_d >= 0.9:
        effect_size_interpretation = "Very Large"
    return effect_size_interpretation

In [29]:
def two_sample_data_setup_for_cohens_d(mean_1, mean_2, sample_std_dev_1, sample_std_dev_2):
    """
    Creates two normal distributions of data values. 
    Calculates differences in values and standard deviation of list of differences.
    Calculates Cohen's D - equation used in dependent t-tests

    :param mean_1: mean of group one
    :param mean_2: mean of group two
    :param sample_std_dev_1: sample standard deviation of group one
    :param sample_std_dev_2: sample standard deviation of group two
    
    :returns: values_1, values_2, cohens_d: values for group one, values for group two, Cohen's d
    """
    values_1 = np.random.normal(loc=mean_1, scale=sample_std_dev_1, size=3000)
    values_2 = np.random.normal(loc=mean_2, scale=sample_std_dev_2, size=3000)
    differences_group_values = np.subtract(values_1, values_2)
    std_dev_differences_group_values = np.std(differences_group_values, ddof=1)
    cohens_d = round(abs((mean_1 - mean_2)/std_dev_differences_group_values), 2)
    return values_1, values_2, cohens_d

In [30]:
import seaborn as sns

def visualize_distributions(values_1, values_2, cohens_d, cohens_d_interpretation):
    """
    Plots KDE plots of values from values_1 and values_2.
    Inserts Cohen's d value and interpretation in title

    :param values_1: values in group one
    :param values_2: values in group two 
    :param cohens_d: Cohen's d value
    :param cohens_d_interpretation: text to describe magnitude of effect size
    :returns: plot figure
    """
    plt.figure(figsize=(10, 8))
    sns.distplot(values_1, hist=False, label='Baseline')
    sns.distplot(values_2, hist=False, label='Recovery')
    plt.xlabel("Log-transformed duration", labelpad=13)
    plt.ylabel("frequency", labelpad=13)
    plt.legend()
    plt.title("Distribution of Two Groups with {0} Cohen's d of {1}".format(cohens_d_interpretation, cohens_d), y=1.015, fontsize=22)

In [31]:
values_bl, values_rec, cohens_d = two_sample_data_setup_for_cohens_d(np.log(ripples_bl.duration).mean(), 
                                                                     np.log(ripples_rec.duration).mean(), 
                                                                     np.log(ripples_bl.duration).std(), 
                                                                     np.log(ripples_rec.duration).std())
visualize_distributions(np.log(ripples_bl.duration), np.log(ripples_rec).duration, cohens_d, interpret_cohens_d(cohens_d))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)


In [32]:
stat, p, med, tbl = stats.median_test(ripples_bl.duration, ripples_rec.duration) 
print(stat, p, med)

10.563289034227415 0.001153557579332585 0.0667999999998301


In [33]:
stats.kruskal(ripples_bl.duration, ripples_rec.duration)

KruskalResult(statistic=17.35478514707542, pvalue=3.101164596042126e-05)

In [34]:
stats.mannwhitneyu(ripples_bl.duration, ripples_rec.duration)

MannwhitneyuResult(statistic=662275.5, pvalue=1.5507795591667666e-05)

## Look at non-normally distributed features

### Max_amplitude

In [35]:
plot_acf_pacf(ripples_bl, ripples_rec, 'max_amplitude')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
plot_vs_time(ripples_bl, ripples_rec, 'max_amplitude')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
test_slope(ripples_bl, 'center_time', 'max_amplitude')

LinregressResult(slope=0.00028648554567230807, intercept=327.2843798925568, rvalue=0.005915418905968726, pvalue=0.8407701887549467, stderr=0.0014256296099204558)

In [38]:
test_slope(ripples_rec, 'center_time', 'max_amplitude')

LinregressResult(slope=-0.004980317096653799, intercept=358.8599671826256, rvalue=-0.12490774722601326, pvalue=8.027237344644863e-06, stderr=0.0011109470653533967)

In [39]:
stats.mannwhitneyu(ripples_bl.max_amplitude, ripples_rec.max_amplitude)

MannwhitneyuResult(statistic=659241.0, pvalue=7.061461402851082e-06)

### Envelope Peak

In [40]:
plot_acf_pacf(ripples_bl, ripples_rec, 'envelope_peak')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
plot_vs_time(ripples_bl, ripples_rec, 'envelope_peak')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
test_slope(ripples_bl, 'center_time', 'envelope_peak')

LinregressResult(slope=0.0017325238816210554, intercept=370.73440302416805, rvalue=0.029766186867209325, pvalue=0.3119305158096863, stderr=0.00171261852802105)

In [43]:
test_slope(ripples_rec, 'center_time', 'envelope_peak')

LinregressResult(slope=-0.006230516600712992, intercept=403.8341854249089, rvalue=-0.1373326757349791, pvalue=8.987330999965224e-07, stderr=0.001261990197286925)

In [44]:
stats.mannwhitneyu(ripples_bl.envelope_peak, ripples_rec.envelope_peak)

MannwhitneyuResult(statistic=702201.0, pvalue=0.032240166889960446)

In [45]:
plt.figure(figsize=(10, 8))
sns.distplot(ripples_bl.envelope_peak, hist=False, label='Baseline')
sns.distplot(ripples_rec.envelope_peak, hist=False, label='Recovery')
plt.xlabel("Envelope peak", labelpad=13)
plt.ylabel("frequency", labelpad=13)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)


### Envelope Integral

In [46]:
plot_acf_pacf(ripples_bl, ripples_rec, 'envelope_integral')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
plot_vs_time(ripples_bl, ripples_rec, 'envelope_integral')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
test_slope(ripples_bl, 'center_time', 'envelope_integral')

LinregressResult(slope=0.2840259769791851, intercept=39641.775156843236, rvalue=0.03161942525617308, pvalue=0.282748295429216, stderr=0.2642919751652511)

In [49]:
test_slope(ripples_rec, 'center_time', 'envelope_integral')

LinregressResult(slope=-0.5572242259851721, intercept=41308.57290515788, rvalue=-0.07663743854313859, pvalue=0.0062862749666023855, stderr=0.20358709794281143)

In [50]:
stats.mannwhitneyu(ripples_bl.envelope_integral, ripples_rec.envelope_integral)

MannwhitneyuResult(statistic=695560.0, pvalue=0.012733605053050722)

In [51]:
plt.figure(figsize=(10, 8))
sns.distplot(ripples_bl.envelope_integral, hist=False, label='Baseline')
sns.distplot(ripples_rec.envelope_integral, hist=False, label='Recovery')
plt.xlabel("Envelope integral", labelpad=13)
plt.ylabel("frequency", labelpad=13)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)


## Look at ripple density

In [52]:
plt.figure(figsize=(10, 8))
sns.distplot(bl_hypno[bl_hypno.state == "N2"].duration, hist=False, label='Baseline')
sns.distplot(rec_hypno[rec_hypno.state == "N2"].duration, hist=False, label='Recovery')
plt.xlabel("NREM Episode Duration", labelpad=13)
plt.ylabel("frequency", labelpad=13)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)


In [53]:
def get_ripple_density(hypno, ripples):  
    N2 = hypno[hypno.state == "N2"]
    N2['n_ripples'] = N2.apply(lambda episode: np.sum(np.logical_and(ripples.center_time >= episode.start_time * 1000, ripples.center_time <= episode.end_time * 1000)), axis=1)
    N2['ripple_density'] = N2.apply(lambda episode: episode.n_ripples / episode.duration, axis=1)
    return N2

In [54]:
bl_density = get_ripple_density(bl_hypno, ripples_bl)

plt.figure()
ax = bl_density.ripple_density.plot.hist(bins=20)

print("Mean ripple density: ", bl_density.ripple_density.mean(), "\n", "Median ripple density: ", bl_density.ripple_density.median())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean ripple density:  0.0 
 Median ripple density:  0.0


In [56]:
ax = bl_density.plot(x='start_time', y='ripple_density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
test_slope(bl_density, 'start_time', 'ripple_density')

LinregressResult(slope=-4.173832579191548e-06, intercept=0.8484553011113574, rvalue=-0.03186301914603328, pvalue=0.8058012901898253, stderr=1.690253733077719e-05)

In [58]:
rec_density = get_ripple_density(rec_hypno, ripples_rec)

plt.figure()
ax = rec_density.ripple_density.plot.hist(bins=20)

print("Mean ripple density: ", rec_density.ripple_density.mean(), "\n", "Median ripple density: ", rec_density.ripple_density.median())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean ripple density:  0.9287416488127143 
 Median ripple density:  0.9719696969696969


In [59]:
plt.figure()
rec_density.boxplot(column=['ripple_density'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [60]:
ax = rec_density.plot(x='start_time', y='ripple_density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [61]:
test_slope(rec_density, 'start_time', 'ripple_density')

LinregressResult(slope=-2.857960321366003e-05, intercept=1.032192804641197, rvalue=-0.2035826851493785, pvalue=0.2481631224433183, stderr=2.4296777687959012e-05)

In [62]:
plot_qq(bl_density, rec_density, 'ripple_density')

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/numpy/lib/function_base.py:2474: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]


In [63]:
stats.ttest_ind(bl_density.ripple_density, rec_density.ripple_density, equal_var=False)

Ttest_indResult(statistic=-1.6143277836765124, pvalue=0.11299018971037275)

In [64]:
values_bl, values_rec, cohens_d = two_sample_data_setup_for_cohens_d(bl_density.ripple_density.mean(), 
                                                                     rec_density.ripple_density.mean(), 
                                                                     bl_density.ripple_density.std(), 
                                                                     rec_density.ripple_density.std())
visualize_distributions(bl_density.ripple_density, rec_density.ripple_density, cohens_d, interpret_cohens_d(cohens_d))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)


In [65]:
stat, p, med, tbl = stats.median_test(bl_density.ripple_density, rec_density.ripple_density) 
print(stat, p, med)

3.688804554079696 0.05477886431560443 0.8416666666666667


In [66]:
stats.kruskal(bl_density.ripple_density, rec_density.ripple_density)

KruskalResult(statistic=5.482663608228726, pvalue=0.01920597156083646)

In [67]:
stats.mannwhitneyu(bl_density.ripple_density, rec_density.ripple_density)

MannwhitneyuResult(statistic=748.5, pvalue=0.009702016125195995)

## Epoched ripple density

In [68]:
# https://stackoverflow.com/questions/21160134/flatten-a-column-with-value-of-type-list-while-duplicating-the-other-columns-va
def unnest_df(df, col, reset_index=False):
    import pandas as pd
    col_flat = pd.DataFrame([[i, x] 
                       for i, y in df[col].apply(list).iteritems() 
                           for x in y], columns=['I', col])
    col_flat = col_flat.set_index('I')
    df = df.drop(col, 1)
    df = df.merge(col_flat, left_index=True, right_index=True)
    if reset_index:
        df = df.reset_index(drop=True)
    return df

def _get_ripple_density(hypno, ripples, epoch_length):  
    N2 = hypno[np.logical_and(hypno.state == "N2", hypno.duration >= epoch_length)]
    N2['epoch_start_times'] = N2.apply(lambda episode: np.arange(episode.start_time, episode.end_time - epoch_length, epoch_length), axis=1)
    N2 = unnest_df(N2, 'epoch_start_times')
    N2 = N2.rename(columns={'end_time': 'episode_end_time', 'start_time': 'epsiode_start_time', 'duration': 'episode_duration', 'epoch_start_times': 'start_time'})
    N2['end_time'] = N2.apply(lambda epoch: epoch.start_time + epoch_length, axis=1)
    N2['n_ripples'] = N2.apply(lambda epoch: np.sum(np.logical_and(ripples.center_time >= epoch.start_time * 1000, ripples.center_time <= epoch.end_time * 1000)), axis=1)
    N2['ripple_density'] = N2.apply(lambda epoch: epoch.n_ripples / epoch_length, axis=1)
    return N2

In [86]:
epoch_length = 10

In [87]:
bl_density = _get_ripple_density(bl_hypno, ripples_bl, epoch_length)

plt.figure()
ax = bl_density.ripple_density.plot.hist()

print("Mean ripple density: ", bl_density.ripple_density.mean(), "\n", "Median ripple density: ", bl_density.ripple_density.median(), "\n", "Number of ripples: ", len(bl_density))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean ripple density:  0.8620553359683796 
 Median ripple density:  0.8 
 Number of ripples:  253


In [88]:
ax = bl_density.plot(x='start_time', y='ripple_density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [89]:
test_slope(bl_density, 'start_time', 'ripple_density')

LinregressResult(slope=-3.6780661464657057e-06, intercept=0.8777217679452868, rvalue=-0.017459707100889704, pvalue=0.7822719766730489, stderr=1.3294729799701525e-05)

In [90]:
rec_density = _get_ripple_density(rec_hypno, ripples_rec, epoch_length)

plt.figure()
ax = rec_density.ripple_density.plot.hist()

print("Mean ripple density: ", rec_density.ripple_density.mean(), "\n", "Median ripple density: ", rec_density.ripple_density.median(), "\n", "Number of ripples: ", len(rec_density))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean ripple density:  1.0527777777777778 
 Median ripple density:  1.0 
 Number of ripples:  360


In [91]:
ax = rec_density.plot(x='start_time', y='ripple_density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [92]:
test_slope(rec_density, 'start_time', 'ripple_density')

LinregressResult(slope=-4.072352343353163e-05, intercept=1.2273491657967954, rvalue=-0.2183692401731002, pvalue=2.9227604481441898e-05, stderr=9.618396015441068e-06)

In [93]:
plot_qq(bl_density, rec_density, 'ripple_density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [94]:
stats.ttest_ind(bl_density.ripple_density, rec_density.ripple_density, equal_var=False)

Ttest_indResult(statistic=-7.114631377730988, pvalue=3.477204130057147e-12)

In [95]:
values_bl, values_rec, cohens_d = two_sample_data_setup_for_cohens_d(bl_density.ripple_density.mean(), 
                                                                     rec_density.ripple_density.mean(), 
                                                                     bl_density.ripple_density.std(), 
                                                                     rec_density.ripple_density.std())
visualize_distributions(bl_density.ripple_density, rec_density.ripple_density, cohens_d, interpret_cohens_d(cohens_d))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)


In [96]:
stat, p, med, tbl = stats.median_test(bl_density.ripple_density, rec_density.ripple_density) 
print(stat, p, med)

25.997054394788773 3.4193867465485304e-07 1.0


In [97]:
stats.kruskal(bl_density.ripple_density, rec_density.ripple_density)

KruskalResult(statistic=48.49786383585921, pvalue=3.30648852463529e-12)

In [98]:
stats.mannwhitneyu(bl_density.ripple_density, rec_density.ripple_density)

MannwhitneyuResult(statistic=30567.0, pvalue=1.6559770566120404e-12)

### Sensitivity Analysis

In [99]:
def _get_density_ttest_pval(epoch_length):
    bl_density = _get_ripple_density(bl_hypno, ripples_bl, epoch_length)
    rec_density = _get_ripple_density(rec_hypno, ripples_rec, epoch_length)
    return stats.ttest_ind(bl_density.ripple_density, rec_density.ripple_density, equal_var=False).pvalue

def _get_density_MannWhitney_pval(epoch_length):
    bl_density = _get_ripple_density(bl_hypno, ripples_bl, epoch_length)
    rec_density = _get_ripple_density(rec_hypno, ripples_rec, epoch_length)
    return stats.mannwhitneyu(bl_density.ripple_density, rec_density.ripple_density).pvalue

def _get_density_nEpochs(hypno, ripples, epoch_length):
    bl_density = _get_ripple_density(hypno, ripples_bl, epoch_length)
    rec_density = _get_ripple_density(rec_hypno, ripples_rec, epoch_length)
    return stats.mannwhitneyu(bl_density.ripple_density, rec_density.ripple_density).pvalue

df = pd.DataFrame(data={'epoch_length': np.arange(5, 65, 5)})
df['ttest_pval'] = df.apply(lambda x: _get_density_ttest_pval(x.epoch_length), axis=1)
df['mw_pval'] = df.apply(lambda x: _get_density_MannWhitney_pval(x.epoch_length), axis=1)
df['n_epochs_bl'] = df.apply(lambda x: len(_get_ripple_density(bl_hypno, ripples_bl, x.epoch_length)), axis=1)
df['n_epochs_rec'] = df.apply(lambda x: len(_get_ripple_density(rec_hypno, ripples_rec, x.epoch_length)), axis=1)

In [100]:
df

,epoch_length,ttest_pval,mw_pval,n_epochs_bl,n_epochs_rec
0,5,4.355856e-13,6.627848e-14,540,741
1,10,3.477204e-12,1.655977e-12,253,360
2,15,2.902205e-12,5.338095e-12,161,234
3,20,5.564504e-13,4.198128e-13,110,173
4,25,1.473783e-12,1.516625e-11,89,135
5,30,1.395253e-12,1.736597e-11,67,108
6,35,2.710032e-10,3.704303e-10,51,94
7,40,1.711919e-10,2.676256e-09,44,82
8,45,1.432978e-12,8.155279e-11,39,72
9,50,1.284163e-10,3.808782e-09,35,60


In [101]:
df[['ttest_pval', 'mw_pval', 'epoch_length']].plot(x='epoch_length')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='epoch_length'>

In [102]:
df[['n_epochs_bl', 'n_epochs_rec', 'epoch_length']].plot(x='epoch_length')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='epoch_length'>